<a href="https://colab.research.google.com/github/RAJamol/Next-word-predictor/blob/main/Next_Word_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.2 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/157M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [37]:
ds['train']['text'][9:13]

[" As with previous Valkyira Chronicles games , Valkyria Chronicles III is a tactical role @-@ playing game where players take control of a military unit and take part in missions against enemy forces . Stories are told through comic book @-@ like panels with animated character portraits , with characters speaking partially through voiced speech bubbles and partially through unvoiced text . The player progresses through a series of linear missions , gradually unlocked as maps that can be freely scanned through and replayed as they are unlocked . The route to each story location on the map varies depending on an individual player 's approach : when one option is selected , the other is sealed off to the player . Outside missions , the player characters rest in a camp , where units can be customized and character growth occurs . Alongside the main story missions are character @-@ specific sub missions relating to different squad members . After the game 's completion , additional episode

In [4]:
len(ds['train'])

1801350

In [5]:
train = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1",split='train[:2%]')

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [7]:
tokenizer = Tokenizer()

In [8]:
a = train['text']

In [9]:
corpus = ' '.join(a)

In [10]:
tokenizer.fit_on_texts([corpus])

Convert text data into numerical sequences

In [11]:
sequences = tokenizer.texts_to_sequences([corpus])

In [12]:
sequences[0][0:10]

[3735, 3787, 870, 18308, 77, 3735, 80, 20634, 3787, 762]

Creating input sequences and target

In [13]:
X,y = [],[]
sequence_len = 10

In [14]:
for i in range(len(sequences[0])-sequence_len):
   X.append(sequences[0][i:i+sequence_len])
   y.append(sequences[0][i+sequence_len])


In [15]:
print(len(X),len(y))

1728248 1728248


In [16]:
y[:9]

[23963, 5863, 3735, 2, 1, 4948, 80, 1874, 995]

In [17]:
y[:9]

[23963, 5863, 3735, 2, 1, 4948, 80, 1874, 995]

In [18]:
X = np.array(X)
y=np.array(y).reshape(-1,1)

In [19]:
vocab_size = len(tokenizer.word_index)+1

In [20]:
np.ndim(y)

2

Preparing Data for training

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((X,y))

In [22]:
dataset = dataset.shuffle(buffer_size=10000).batch(64).prefetch(tf.data.AUTOTUNE)

In [23]:
size = sum(1 for _ in dataset)

creating train,test and validation split

In [24]:
train = dataset.take(int(0.8*size))
test = dataset.take(int(0.1*size))
vald= dataset.take(int(0.1*size))

Model

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Input

In [26]:
model = Sequential()
model.add(Input(shape=(10,)))
model.add(Embedding(vocab_size,100, input_length = 10))
model.add(LSTM(150,return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(vocab_size,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 10, 100)             │       6,512,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 10, 150)             │         150,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 10, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 100)                 │         100,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 65129)               │       6,578,029 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,341,929 (50.90 MB)

 Trainable params: 13,341,929 (50.90 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
history = model.fit(train,validation_data = vald,epochs=25,verbose = 1)

Epoch 1/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 503s 23ms/step - accuracy: 0.0939 - loss: 7.4829 - val_accuracy: 0.1252 - val_loss: 7.2470
Epoch 2/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 498s 23ms/step - accuracy: 0.1313 - loss: 6.7747 - val_accuracy: 0.1374 - val_loss: 7.0688
Epoch 3/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 513s 24ms/step - accuracy: 0.1470 - loss: 6.3910 - val_accuracy: 0.1500 - val_loss: 6.9021
Epoch 4/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 500s 23ms/step - accuracy: 0.1601 - loss: 6.0976 - val_accuracy: 0.1601 - val_loss: 6.7102
Epoch 5/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 507s 23ms/step - accuracy: 0.1721 - loss: 5.8764 - val_accuracy: 0.1665 - val_loss: 6.5399
Epoch 6/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 560s 23ms/step - accuracy: 0.1818 - loss: 5.7126 - val_accuracy: 0.1749 - val_loss: 6.4001
Epoch 7/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 499s 23ms/step - accuracy: 0.1907 - loss: 5.5696 - val_accuracy: 0.1813 - val_loss: 6.3165
Epoch 8/25
21603/21603 ━━━━━━━━━━━━━━━━━━━━ 499s 23ms/s

Saving the Model

In [49]:
model.save("model.keras")

Getting output

In [50]:
import numpy as np

def predict_next_word(model, tokenizer, text, sequence_length):
    # Tokenize input text
    tokenized_text = tokenizer.texts_to_sequences([text])[0]

    # Pad/truncate to match sequence length
    tokenized_text = tf.keras.preprocessing.sequence.pad_sequences(
        [tokenized_text], maxlen=sequence_length, padding='pre'
    )

    # Get predictions
    predictions = model.predict(tokenized_text)
    predicted_index = np.argmax(predictions, axis=-1)[0]  # Get highest probability index

    # Convert index to word
    predicted_word = tokenizer.index_word.get(predicted_index, "<UNK>")

    return predicted_word

# Example prediction
input_text = "After a long day, she felt very "
next_word = predict_next_word(model, tokenizer, input_text, 10)
print(f"Predicted next word: {next_word}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Predicted next word: much
